# Estimate Pi interactively!

Below you'll see a few widget windows, these run a simulation that estimates the value of "Pi" based on the number of random points that fall within vs. outside of a circle. Try playing around with the number of total samples to see how the estimate stabilizes over time!

In [1]:
from bqplot import pyplot as plt
import numpy as np
# import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets.widgets import IntSlider, HBox, Button
%matplotlib inline

In [2]:
def circ(x, radius=2):
    y = np.sqrt(radius**2 - x**2)
    return y

def calculate_ratios(samples):

    samples_y_curve = circ(samples[:, 0])
    inside_dots = samples_y_curve > samples[:, 1]

    ratios = []
    for n_plt in range(N_SAMPLES):
        if n_plt == 0:
            ratios.append(0)
            continue
        ratios.append(4 * (sum(inside_dots[:n_plt]) / len(inside_dots[:n_plt])))

    return samples_y_curve, inside_dots, ratios

def _update_plot(change=None):
    n_plt = slider.value
    plt_samples = plot_data['data'][:n_plt]
    inside_dots = plot_data['inside']
    ratios = plot_data['ratios']

    colors = np.where(inside_dots[:n_plt], '#41f465', "#f44b42").tolist()
    scat.x = plt_samples[:, 0]
    scat.y = plt_samples[:, 1]
    scat.colors = colors
    fig.title = 'Ratio of in-to-out dots: {}'.format(ratios[n_plt])
    
    fig_line.x = range(n_plt)
    line.y = ratios[:n_plt]
    line.x = range(n_plt)
    
def _reset_data(change=None):
    samples = np.random.random(N_SAMPLES * 2) * 2
    samples = samples.reshape([-1, 2])
    
    samples_y_curve, inside_dots, ratios = calculate_ratios(samples)
    
    plot_data['data'] = samples
    plot_data['inside'] = inside_dots
    plot_data['ratios'] = ratios
    _update_plot()

In [3]:
N_SAMPLES = 700
samples = np.random.random(N_SAMPLES * 2) * 2
samples = samples.reshape([-1, 2])
samples_y_curve, inside_dots, ratios = calculate_ratios(samples)

In [21]:
width = '350px'

In [20]:
plt.clear()

plot_data = {'data': samples, 'ratios': ratios, 'inside': inside_dots}

fig = plt.figure()
x = np.linspace(0, 5, 1000)
y = circ(x)
myplt = plt.plot(x=x, y=y, s=[100] * len(y))
scat = plt.scatter(samples[:10, 0], samples[:10, 1], options={'s': 1})
plt.xlim(0, 2)
fig.layout.height = width
fig.layout.width = width

fig_line = plt.figure()
fig_line.layout.height = width
fig_line.layout.width = width
fig_line.title = "Estimation of Pi"
line = plt.plot(range(10), ratios[:10])
line2 = plt.hline(np.pi, ls='--', c='k')
plt.xlim(0, N_SAMPLES)
plt.ylim(2, 4)

slider = IntSlider(value=10, min=1, max=N_SAMPLES-1, description="$N_{samples}$")
slider.observe(_update_plot)

button = Button(description="Reset data")
button.on_click(_reset_data)

_update_plot()

box = HBox([fig, fig_line])
box2 = HBox([slider, button])
display(box, box2)

/home/choldgraf/anaconda/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in sqrt
  
/home/choldgraf/anaconda/envs/dev/lib/python3.6/site-packages/bqplot/pyplot.py:602: FutureWarning: Conversion of the second argument of issubdtype from `str` to `str` is deprecated. In future, it will be treated as `np.str_ == np.dtype(str).type`.
  issubdtype(dtype, Scale.scale_types[key].dtype)
/home/choldgraf/anaconda/envs/dev/lib/python3.6/site-packages/bqplot/traits.py:137: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(a.dtype, np.float):


HBox(children=(Figure(axes=[Axis(scale=LinearScale(max=2.0, min=0.0)), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='300px', min_width='125px', width='300px'), marks=[Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=2.0, min=0.0), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([0.        , 0.00500501, 0.01001001, 0.01501502, 0.02002002,
       0.02502503, 0.03003003, 0.03503504, 0.04004004, 0.04504505,
       0.05005005, 0.05505506, 0.06006006, 0.06506507, 0.07007007,
       0.07507508, 0.08008008, 0.08508509, 0.09009009, 0.0950951 ,
       0.1001001 , 0.10510511, 0.11011011, 0.11511512, 0.12012012,
       0.12512513, 0.13013013, 0.13513514, 0.14014014, 0.14514515,
       0.15015015, 0.15515516, 0.16016016, 0.16516517, 0.17017017,
       0.17517518, 0.18018018, 0.18518519, 0.19019019, 0.1951952 ,
       0.2002002 , 0.20520521, 0.21021021, 0.21521522, 0.22022022,
       0.22522523, 0.23023023, 0.23523524, 0.24024024, 0.24524525,
       0.25025025, 0.25525526, 0.26026026, 0.26526527, 0.27027027,
       0.27527528, 0.28028028, 0.28528529, 0.29029029, 0.2952953 ,
       0.3003003 , 0.30530531, 0.31031031, 0.31531532, 0.32032032,
       0.32532533, 0.33033033, 0.33533534, 0.34034034, 0.34534535,
       0.35035035, 0.35535536, 0.36036036, 0.36536537, 0.37037037,
       0.37537538, 0.38038038, 0.38538539, 0.39039039, 0.3953954 ,
       0.4004004 , 0.40540541, 0.41041041, 0.41541542, 0.42042042,
       0.42542543, 0.43043043, 0.43543544, 0.44044044, 0.44544545,
       0.45045045, 0.45545546, 0.46046046, 0.46546547, 0.47047047,
       0.47547548, 0.48048048, 0.48548549, 0.49049049, 0.4954955 ,
       0.5005005 , 0.50550551, 0.51051051, 0.51551552, 0.52052052,
       0.52552553, 0.53053053, 0.53553554, 0.54054054, 0.54554555,
       0.55055055, 0.55555556, 0.56056056, 0.56556557, 0.57057057,
       0.57557558, 0.58058058, 0.58558559, 0.59059059, 0.5955956 ,
       0.6006006 , 0.60560561, 0.61061061, 0.61561562, 0.62062062,
       0.62562563, 0.63063063, 0.63563564, 0.64064064, 0.64564565,
       0.65065065, 0.65565566, 0.66066066, 0.66566567, 0.67067067,
       0.67567568, 0.68068068, 0.68568569, 0.69069069, 0.6956957 ,
       0.7007007 , 0.70570571, 0.71071071, 0.71571572, 0.72072072,
       0.72572573, 0.73073073, 0.73573574, 0.74074074, 0.74574575,
       0.75075075, 0.75575576, 0.76076076, 0.76576577, 0.77077077,
       0.77577578, 0.78078078, 0.78578579, 0.79079079, 0.7957958 ,
       0.8008008 , 0.80580581, 0.81081081, 0.81581582, 0.82082082,
       0.82582583, 0.83083083, 0.83583584, 0.84084084, 0.84584585,
       0.85085085, 0.85585586, 0.86086086, 0.86586587, 0.87087087,
       0.87587588, 0.88088088, 0.88588589, 0.89089089, 0.8958959 ,
       0.9009009 , 0.90590591, 0.91091091, 0.91591592, 0.92092092,
       0.92592593, 0.93093093, 0.93593594, 0.94094094, 0.94594595,
       0.95095095, 0.95595596, 0.96096096, 0.96596597, 0.97097097,
       0.97597598, 0.98098098, 0.98598599, 0.99099099, 0.995996  ,
       1.001001  , 1.00600601, 1.01101101, 1.01601602, 1.02102102,
       1.02602603, 1.03103103, 1.03603604, 1.04104104, 1.04604605,
       1.05105105, 1.05605606, 1.06106106, 1.06606607, 1.07107107,
       1.07607608, 1.08108108, 1.08608609, 1.09109109, 1.0960961 ,
       1.1011011 , 1.10610611, 1.11111111, 1.11611612, 1.12112112,
       1.12612613, 1.13113113, 1.13613614, 1.14114114, 1.14614615,
       1.15115115, 1.15615616, 1.16116116, 1.16616617, 1.17117117,
       1.17617618, 1.18118118, 1.18618619, 1.19119119, 1.1961962 ,
       1.2012012 , 1.20620621, 1.21121121, 1.21621622, 1.22122122,
       1.22622623, 1.23123123, 1.23623624, 

HBox(children=(IntSlider(value=10, description='$N_{samples}$', max=699, min=1), Button(description='Reset data', style=ButtonStyle())))